<div style="text-align: center; margin-bottom: 15px;">
        <svg viewBox="0 0 1200 250" xmlns="http://www.w3.org/2000/svg" style="max-width: 100%; height: auto;">
            <!-- Background -->
            <rect x="0" y="0" width="1200" height="250" rx="10" fill="#f8f9fa" />
            <!-- Stylized myotube with nuclei - extended across header -->
            <path d="M50,125 C150,95 250,85 350,90 C450,95 550,115 650,120 C750,125 850,115 950,110 C1050,105 1100,125 1150,125 C1100,145 1050,155 950,150 C850,145 750,135 650,140 C550,145 450,165 350,160 C250,155 150,145 50,125 Z" fill="#8bc34a" fill-opacity="0.7" stroke="#4caf50" stroke-width="2" />
            <!-- Nuclei inside the myotube -->
            <circle cx="200" cy="125" r="15" fill="#2c5aa0" />
            <circle cx="400" cy="125" r="15" fill="#2c5aa0" />
            <circle cx="600" cy="125" r="15" fill="#2c5aa0" />
            <circle cx="800" cy="125" r="15" fill="#2c5aa0" />
            <circle cx="1000" cy="125" r="15" fill="#2c5aa0" />
            <!-- Targeting reticle over side nucleus -->
            <circle cx="800" cy="125" r="30" fill="none" stroke="#ff5722" stroke-width="1.5" stroke-dasharray="5,3" />
            <line x1="800" y1="85" x2="800" y2="165" stroke="#ff5722" stroke-width="1.5" />
            <line x1="760" y1="125" x2="840" y2="125" stroke="#ff5722" stroke-width="1.5" />
            <!-- MUSCLE Text - positioned above myotube -->
            <text x="600" y="65" font-family="Arial, sans-serif" font-size="60" font-weight="bold" text-anchor="middle" fill="#2c5aa0">MUSCLE</text>
            <!-- Subtitle text - larger and more prominent -->
            <text x="600" y="190" font-family="Arial, sans-serif" font-size="24" font-weight="bold" text-anchor="middle" fill="#555">Myotube and Nuclei Skeletal Cell Locator &amp; Evaluator</text>
        </svg>
</div>


This notebook provides a comprehensive suite for analyzing microscopy images to detect and quantify myotubes and nuclei. It calculates various metrics, including myotube size, nuclei count per myotube, and the Nuclear Fusion Index (NFI).

---

## 🧪 Workflow Overview

- **Image Loading & Preprocessing**: Loads images and selects relevant color channels.  
- **Nuclei Diameter Estimation**: Automatically estimates the average diameter of nuclei, crucial for Cellpose segmentation, or allows manual input.  
- **Myotube & Nuclei Detection**: Uses Cellpose for nuclei segmentation and a custom algorithm for myotube segmentation. This includes an optimization step to refine myotube masks based on coverage and nuclei intensity.  
- **Quantitative Analysis**: Measures properties of detected myotubes (length, diameter, area, nuclei count) and calculates overall metrics like NFI.  
- **Visualization & Saving**: Generates visual outputs (original image, masks, overlays) and saves plots, masks, and quantitative results to files.  

---

## 🚀 How to Use

- **Interactive Analysis (UI)**: Run all cells in this notebook. An interactive widget-based UI will appear at the bottom, allowing you to specify input/output directories, adjust parameters, and run the analysis on a dataset of images.  
- **Programmatic Use (for Researchers)**: The functions defined in this notebook can be called individually if you prefer to integrate parts of this pipeline into your own scripts or conduct more fine-grained experiments. Each section of functions is preceded by a Markdown cell explaining its purpose and parameters.

### 1. Import Libraries

In [ ]:
import os
import numpy as np
import cv2
import torch
from skimage import io, measure
from skimage.draw import polygon_perimeter
from skimage.util import img_as_float
from skimage.transform import resize
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from cellpose import models
import tifffile
from matplotlib.colors import ListedColormap
from ipywidgets import widgets, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text, Checkbox, VBox, HBox, Output
from IPython.display import display, HTML, clear_output
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import time
import ipywidgets as widgets

#### Core Libraries

This notebook relies on several key Python libraries:

- **NumPy**: For numerical operations, especially array manipulation.  
- **OpenCV (`cv2`)**: For image processing tasks like thresholding and morphological operations.  
- **scikit-image (`skimage`)**: For image analysis, including region property measurements and transformations.  
- **Matplotlib**: For plotting and visualizing images and results.  
- **PyTorch**: As a backend for Cellpose, particularly if using a GPU.  
- **Cellpose**: A deep learning-based segmentation tool, used here for nuclei detection.  
- **tifffile**: For reading and writing TIFF image files, often used in microscopy.  
- **ipywidgets & IPython.display**: To create the interactive user interface within the Jupyter Notebook.  
- **Pandas & openpyxl**: For organizing and saving results in tabular formats like CSV and Excel.

### 2. Image Loading and Preprocessing

Load and preprocess images by selecting appropriate color channels for analysis.

In [ ]:


def load_image(image_path):
    return io.imread(image_path)

def preprocess(image, myotube_channel='green'):
    if image.ndim == 2:
        selected_channel = image
        blue_channel = image
    else:
        channel_idx = {'red': 0, 'green': 1}
        selected_channel = image[:, :, channel_idx[myotube_channel]]
        blue_channel = image[:, :, 2]  # blue channel for nuclei detection
    return selected_channel, blue_channel

#### Image Loading and Preprocessing

The `load_image()` function imports microscopy images while preserving their original properties.

The `preprocess()` function extracts relevant channels:
- For myotubes: Uses green channel (default) or red channel
- For nuclei: Uses blue channel, where nuclear stains (DAPI, Hoechst) typically appear

This channel separation enables targeted detection of different biological structures in subsequent steps.

### 3. Diameter Estimation

Estimate the diameter of nuclei either manually or automatically.

In [ ]:
def estimate_diameter(blue_channel, use_cropping=True, use_manual=False, manual_value=17.0, output_widget=None):
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    # If manual diameter is specified, return it directly
    if use_manual:
        update_output(f"Using manual diameter: {manual_value:.2f}")
        return manual_value
    
    # Prepare image
    if blue_channel.ndim == 3:
        img_for_size = blue_channel[0] if blue_channel.shape[0] == 1 else blue_channel[:,:,0]
    elif blue_channel.ndim == 2:
        img_for_size = blue_channel
    else:
        return 17.0  # Default fallback
    
    try:
        # Create a smaller view to analyze by cropping instead of resizing
        if use_cropping:
            # Get the center region of the image (30% of each dimension)
            h, w = img_for_size.shape
            crop_h, crop_w = int(h * 0.3), int(w * 0.3)
            start_h, start_w = (h - crop_h) // 2, (w - crop_w) // 2
            
            # Crop the image
            cropped_img = img_for_size[start_h:start_h+crop_h, start_w:start_w+crop_w]
            update_output(f"Using cropped region of size {crop_h}x{crop_w} for estimation")
            
            # Load CPU model for diameter estimation
            diameter_model = models.Cellpose(model_type='nuclei', gpu=False)
            
            # Get diameter estimate directly from cropped region
            estimated_diameter, _ = diameter_model.sz.eval(cropped_img, channels=[0, 0])
            
            # No need to rescale since I used actual pixels
            update_output(f"Estimated nuclei diameter: {estimated_diameter:.2f}")
        else:
            # Use the original image (slower but most accurate)
            diameter_model = models.Cellpose(model_type='nuclei', gpu=False)
            estimated_diameter, _ = diameter_model.sz.eval(img_for_size, channels=[0, 0])
            update_output(f"Estimated nuclei diameter from full image: {estimated_diameter:.2f}")
        
        # Clean up
        del diameter_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        return estimated_diameter
        
    except Exception as e:
        update_output(f"Error during diameter estimation: {e}")
        update_output("Using default diameter of 17.0")
        return 17.0

#### Nuclei Diameter Estimation

This function determines the average nuclei diameter - a critical parameter for Cellpose segmentation:

- **Manual mode**: Uses a fixed user-provided diameter value (default: 17.0 pixels)
- **Automatic mode**: Leverages Cellpose's built-in estimator on either:
  - A cropped central region (30% of image area) for faster processing
  - The full image for maximum accuracy

The diameter value directly impacts segmentation quality - underestimation leads to merged nuclei, while overestimation causes fragmentation.

### 4. Myotube and Nuclei Detection

Detect myotubes and nuclei using intensity thresholds and Cellpose segmentation.

In [ ]:
def detect_myotubes(green_channel, blue_channel, image, use_gpu, min_myotube_size, max_coverage_percent,
                    minimum_fiber_intensity, maximum_fiber_intensity, 
                    initial_nuclei_threshold, max_nuclei_threshold, use_manual_diameter=False, manual_diameter=17.0, use_cropping=True, output_widget=None):
    # Update output widget instead of print
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    update_output("Detecting myotubes and nuclei...")

    # Check if CUDA is available
    use_gpu = use_gpu and torch.cuda.is_available()
    if use_gpu:
        update_output("CUDA is available. Using GPU for Cellpose.")
    else:
        update_output("Using CPU for Cellpose.")

    def _get_fiber_mask(fiber_channel, nuclei_channel, minimum_intensity, maximum_intensity, nuclei_threshold):
        # First, apply a base threshold
        kernel = np.ones((4, 4), np.uint8)
        _, min_thresh = cv2.threshold(fiber_channel, minimum_intensity, 255, cv2.THRESH_BINARY)
        _, max_thresh = cv2.threshold(fiber_channel, maximum_intensity, 255, cv2.THRESH_BINARY)
        processed = min_thresh - max_thresh
        
        # Opening, to remove noise in the background
        processed = cv2.morphologyEx(processed, cv2.MORPH_OPEN, kernel)
        
        # Closing, to remove noise inside the fibers
        processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)
        
        # Inverting black and white
        processed = cv2.bitwise_not(processed)
        
        # Find contours of the holes inside the fibers
        contours, _ = cv2.findContours(processed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            mask = np.zeros_like(processed)
            cv2.drawContours(mask, [contour], -1, 255, -1)
            
            if np.average(nuclei_channel[mask == 255]) > nuclei_threshold:
                cv2.drawContours(processed, [contour], -1, 0, -1)
        
        # Inverting black and white again
        processed = cv2.bitwise_not(processed)
        
        # Creating a boolean mask
        mask = processed.astype(bool)
        
        return mask

    def calculate_coverage(mask):
        return np.sum(mask) / mask.size * 100
    
    # --- START: Modified Diameter Estimation ---
    estimated_diameter = estimate_diameter(
        blue_channel, 
        use_cropping=use_cropping,
        use_manual=use_manual_diameter, 
        manual_value=manual_diameter,
        output_widget=output_widget
    )
    # --- END: Modified Diameter Estimation ---

    model = models.Cellpose(model_type='nuclei', gpu=use_gpu)
    nuclei_masks, _, _, _ = model.eval(
    blue_channel,
    diameter=estimated_diameter,
    flow_threshold=0.4,
    cellprob_threshold=0.0,
    channels=[0, 0],
    normalize=True
    )
                                            
    
    # Create a binary mask for nuclei
    nuclei_binary = nuclei_masks > 0
    nuclei_count = np.sum(nuclei_binary) #delete later
    print(f"DEBUG - Initial nuclei count (raw from Cellpose): {nuclei_count}") #delete later
    print(f"DEBUG - Blue channel shape: {blue_channel.shape}") #delete later
    print(f"DEBUG - Blue channel min/max: {np.min(blue_channel)}/{np.max(blue_channel)}") #delete later
    # Dilate nuclei slightly to ensure overlap with myotubes
    nuclei_dilated = cv2.dilate(nuclei_binary.astype(np.uint8), cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9)))

    nuclei_threshold = initial_nuclei_threshold
    
    update_output("Optimizing myotube detection...")
    while True:
        # Get the fiber mask
        myotube_mask = _get_fiber_mask(green_channel, blue_channel, minimum_fiber_intensity, maximum_fiber_intensity, nuclei_threshold)

        # Remove small objects
        myotube_mask = cv2.morphologyEx(myotube_mask.astype(np.uint8), cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(myotube_mask, connectivity=8)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] < min_myotube_size:
                myotube_mask[labels == i] = 0

        # Fill holes with overlapping nuclei
        overlapping_nuclei = nuclei_dilated & cv2.dilate(myotube_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
        myotube_mask = myotube_mask | overlapping_nuclei

        # Calculate coverage
        coverage = calculate_coverage(myotube_mask)
        
        update_output(f"Current nuclei threshold: {nuclei_threshold}, Coverage: {coverage:.2f}%")

        if coverage <= max_coverage_percent or nuclei_threshold >= max_nuclei_threshold:
            break
        
        nuclei_threshold += 20
        nuclei_threshold = min(nuclei_threshold, max_nuclei_threshold)

    # Label the myotubes
    labeled = measure.label(myotube_mask)
    
    update_output("Myotube detection complete.")
    return labeled, coverage, nuclei_masks

#### Myotube and Nuclei Detection

This function performs dual segmentation of myotubes and nuclei:

- **Nuclei Detection**: Uses Cellpose with the estimated diameter parameter to accurately segment nuclei
- **Myotube Detection**: Employs intensity-based thresholding with morphological refinement
- **Optimization Loop**: Iteratively adjusts nuclei intensity threshold until myotube coverage reaches target percentage
- **Spatial Integration**: Ensures proper association between nuclei and myotubes through dilated overlapping regions

This approach balances sensitivity (detecting true myotubes) with specificity (avoiding false positives), critical for accurate morphological and fusion index calculations.

### 5. Analysis and Metrics Calculation

Analyze detected myotubes and calculate metrics such as Nuclear Fusion Index.

In [ ]:
def analyze_myotubes(myotube_mask, nuclei_mask):
    # Get properties of labeled regions
    myotube_props = measure.regionprops(myotube_mask)
    
    # Label the nuclei
    labeled_nuclei = measure.label(nuclei_mask)
    
    myotube_data = []
    total_nuclei = np.max(labeled_nuclei)
    nuclei_in_myotubes = 0
    
    for prop in myotube_props:
        # Create a mask for the current myotube
        current_myotube_mask = myotube_mask == prop.label
        
        # Count nuclei inside the myotube
        nuclei_in_myotube = np.unique(labeled_nuclei[current_myotube_mask])
        nuclei_count = len(nuclei_in_myotube) - 1 if 0 in nuclei_in_myotube else len(nuclei_in_myotube)
        
        # Only process myotubes with at least 3 nuclei
        if nuclei_count >= 3:
            nuclei_in_myotubes += nuclei_count
            
            # Calculate length (major axis length)
            length = prop.major_axis_length
            
            # Calculate diameter (minor axis length)
            diameter = prop.minor_axis_length
            
            # Calculate surface area
            surface_area = prop.area
            
            # Append data for this myotube
            myotube_data.append({
                'id': prop.label,
                'nuclei_count': nuclei_count,
                'length': length,
                'diameter': diameter,
                'surface_area': surface_area
            })
    
    # Calculate Nuclear Fusion Index
    nuclear_fusion_index = (nuclei_in_myotubes / total_nuclei) * 100 if total_nuclei > 0 else 0
    
    return myotube_data, total_nuclei, nuclei_in_myotubes, nuclear_fusion_index

#### Myotube Analysis and Fusion Metrics

This function quantifies key morphological and developmental parameters:

- Extracts metrics for each myotube: length, diameter, surface area
- Counts nuclei within each myotube (filtering for those with ≥3 nuclei)
- Calculates the Nuclear Fusion Index (NFI) = (nuclei in myotubes / total nuclei) × 100

NFI is a critical metric for assessing myoblast differentiation efficiency and is widely used in muscle development and disease studies.

### 6. Visualization

Visualize analysis results through colored masks and overlays.

In [ ]:
def create_colored_myotube_mask(myotube_mask):
    # Label each myotube uniquely
    labeled_myotubes = measure.label(myotube_mask)
    
    # Generate a colormap with unique colors for each label
    num_labels = np.max(labeled_myotubes)
    colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, num_labels + 1))
    colors[0] = [1, 1, 1, 1]  # Set background to white
    
    # Create a new colormap
    cmap = ListedColormap(colors)
    
    # Apply the colormap to the labeled myotubes
    colored_mask = cmap(labeled_myotubes)
    
    return (colored_mask * 255).astype(np.uint8)

def create_myotube_overlay(image, myotube_mask):
    # Create a colored overlay for myotubes
    overlay = image.copy()
    if overlay.ndim == 2:
        from skimage import color
        overlay = color.gray2rgb(overlay)
    
    # Find all contours using OpenCV, including internal ones
    contours, hierarchy = cv2.findContours(myotube_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw all contours on the overlay
    cv2.drawContours(overlay, contours, -1, (255, 0, 0), 2)
    
    return overlay

def create_nuclei_visualization(nuclei_mask, myotube_mask):
    # Create a visualization of nuclei
    nuclei_viz = np.zeros((*nuclei_mask.shape, 3), dtype=np.uint8)
    
    # Label nuclei
    labeled_nuclei = measure.label(nuclei_mask)
    
    # Identify nuclei inside myotubes
    nuclei_in_myotubes = labeled_nuclei * (myotube_mask > 0)
    
    # Color nuclei inside myotubes green, others red
    nuclei_viz[nuclei_in_myotubes > 0] = [0, 255, 0]  # Green
    nuclei_viz[(labeled_nuclei > 0) & (nuclei_in_myotubes == 0)] = [255, 0, 0]  # Red
    
    return nuclei_viz

def visualize_and_save_results(image, myotube_mask, nuclei_mask, coverage, image_name, total_nuclei, nuclei_in_myotubes, nuclear_fusion_index, save_dir=None, display_fig=True, output_widget=None):
    # Update output
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    update_output(f"Creating visualization for {image_name}...")
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle(f"Analysis Results for {image_name}", fontsize=16)
    
    # Original Image
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    # Myotube Mask
    colored_myotube_mask = create_colored_myotube_mask(myotube_mask)
    ax2.imshow(colored_myotube_mask)
    ax2.set_title(f'Myotube Mask\n(Coverage: {coverage:.2f}%)')
    ax2.axis('off')
    
    # Myotube Overlay
    overlay = create_myotube_overlay(image, myotube_mask)
    ax3.imshow(overlay)
    ax3.set_title('Myotube Overlay')
    ax3.axis('off')
    
    # Nuclei Visualization
    nuclei_viz = create_nuclei_visualization(nuclei_mask, myotube_mask)
    ax4.imshow(nuclei_viz)
    ax4.set_title('Nuclei Visualization')
    ax4.axis('off')

    # Add statistics text explicitly here on the nuclei visualization plot
    stats_text = (f'Total nuclei: {total_nuclei}\n'
                  f'Nuclei in myotubes: {nuclei_in_myotubes}\n'
                  f'Nuclear Fusion Index: {nuclear_fusion_index:.2f}%')

    ax4.text(0.02, 0.98, stats_text, fontsize=12, color='white', 
             verticalalignment='top', bbox=dict(facecolor='black', alpha=0.5),
             transform=ax4.transAxes)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust to make room for the main title
    
    # Display the plot if requested
    if display_fig and output_widget is not None:
        with output_widget:
            display(fig)
    
    # Save the figure if a save directory is provided
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{os.path.splitext(image_name)[0]}_analysis.png")
        fig.savefig(save_path, bbox_inches='tight', facecolor='white', edgecolor='none')
        update_output(f"Plot saved to: {save_path}")
    
    # Don't close the figure when processing is done so it remains visible in the output
    if not display_fig:
        plt.close(fig)  # Only close if not displaying (e.g., when saving only)

#### Visualization Functions

These functions generate informative visual outputs for analysis and interpretation:

- `create_colored_myotube_mask`: Assigns unique colors to each detected myotube for visual distinction
- `create_myotube_overlay`: Highlights myotube boundaries on the original image
- `create_nuclei_visualization`: Colors nuclei based on myotube association (green: inside, red: outside)
- `visualize_and_save_results`: Combines all visualizations with metrics in a 4-panel figure

These visualizations enable both qualitative assessment of segmentation quality and quantitative validation of fusion metrics.

### 7. Data Saving Functions

Functions to save generated masks and analysis results.

In [ ]:
def save_mask_files(myotube_mask, nuclei_mask, save_dir, base_filename, output_widget=None):
    # Update output
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Save myotube mask
    myotube_filename = os.path.join(save_dir, f"{base_filename}_myotubes_masks.tif")
    tifffile.imwrite(myotube_filename, myotube_mask.astype(np.uint16))
    
    # Save nuclei mask
    nuclei_filename = os.path.join(save_dir, f"{base_filename}_nuclei_masks.tif")
    tifffile.imwrite(nuclei_filename, nuclei_mask.astype(np.uint16))
    
    update_output(f"Saved myotube mask to: {myotube_filename}")
    update_output(f"Saved nuclei mask to: {nuclei_filename}")

def process_single_image(image_path, use_gpu, min_myotube_size, max_coverage_percent,
                         minimum_fiber_intensity, maximum_fiber_intensity,
                         initial_nuclei_threshold, max_nuclei_threshold,
                         myotube_channel='green',  # new parameter
                         save_plots=False, plots_dir=None,
                         save_masks=False, masks_dir=None, 
                         use_manual_diameter=False, manual_diameter=17.0, use_cropping=True,
                         output_widget=None):
    # Update output
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    update_output(f"Processing image: {os.path.basename(image_path)}")
    
    image = load_image(image_path)
    selected_channel, blue_channel = preprocess(image, myotube_channel)  # Updated
    myotube_mask, coverage, nuclei_mask = detect_myotubes(
        selected_channel, blue_channel, image, use_gpu, 
        min_myotube_size, max_coverage_percent,
        minimum_fiber_intensity, maximum_fiber_intensity,
        initial_nuclei_threshold, max_nuclei_threshold,
        use_manual_diameter, manual_diameter, use_cropping,
        output_widget
    )
    
    update_output("Analyzing myotubes...")
    myotube_results, total_nuclei, nuclei_in_myotubes, nuclear_fusion_index = analyze_myotubes(myotube_mask, nuclei_mask)
    update_output(f"Total nuclei found: {total_nuclei}")
    update_output(f"Nuclei inside myotubes (with at least 3 nuclei): {nuclei_in_myotubes}")
    update_output(f"Nuclear Fusion Index: {nuclear_fusion_index:.2f}%")
    
    image_name = os.path.basename(image_path)
    
    # Always visualize, but only save if save_plots is True
    visualize_and_save_results(
        image, myotube_mask, nuclei_mask, coverage, image_name, 
        total_nuclei, nuclei_in_myotubes, nuclear_fusion_index, 
        plots_dir if save_plots else None,
        True,  # Always display the figure
        output_widget
    )
    
    if save_masks and masks_dir:
        save_mask_files(myotube_mask, nuclei_mask, masks_dir, os.path.splitext(image_name)[0], output_widget)
    
    return {
        'myotube_results': myotube_results,
        'coverage': coverage,
        'total_nuclei': total_nuclei,
        'nuclei_in_myotubes': nuclei_in_myotubes,
        'nuclear_fusion_index': nuclear_fusion_index
    }


def process_dataset(directory, use_gpu, min_myotube_size, max_coverage_percent, 
                    minimum_fiber_intensity, maximum_fiber_intensity, 
                    initial_nuclei_threshold, max_nuclei_threshold,
                    myotube_channel='green',  # new parameter
                    save_plots=False, plots_dir=None, 
                    save_masks=False, masks_dir=None, 
                    save_results=False, results_dir=None, results_format=None,
                    use_manual_diameter=False, manual_diameter=17.0, use_cropping=True,
                    output_widget=None, progress_widget=None):
    # Update output
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    # Create directories if needed
    if save_plots and plots_dir:
        os.makedirs(plots_dir, exist_ok=True)
    if save_masks and masks_dir:
        os.makedirs(masks_dir, exist_ok=True)
    if save_results and results_dir:
        os.makedirs(results_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(directory) if f.endswith(('.tif', '.tiff', '.jpg', '.jpeg', '.png'))]
    all_results = []

    num_images = len(image_files)
    start_time_total = time.time()
    
    update_output(f"Found {num_images} images to process.")
    
    # Set up progress widget if provided
    if progress_widget is not None:
        progress_widget.max = num_images
        progress_widget.value = 0

    for idx, image_file in enumerate(image_files, start=1):
        image_path = os.path.join(directory, image_file)

        update_output(f"<b>Processing image {idx}/{num_images}: {image_file}</b>")
        start_time = time.time()

        # For the last image in the dataset, we want to keep the plot visible
        # For all others, we can close the figure to save memory
        is_last_image = (idx == num_images)
        
        results = process_single_image(
            image_path, use_gpu, min_myotube_size, max_coverage_percent,
            minimum_fiber_intensity, maximum_fiber_intensity,
            initial_nuclei_threshold, max_nuclei_threshold,
            myotube_channel=myotube_channel,  # pass this through
            save_plots=save_plots, plots_dir=plots_dir,
            save_masks=save_masks, masks_dir=masks_dir,
            use_manual_diameter=use_manual_diameter, 
            manual_diameter=manual_diameter, 
            use_cropping=use_cropping,
            output_widget=output_widget
        )

        elapsed_time = time.time() - start_time
        total_elapsed_time = time.time() - start_time_total
        estimated_total_time = (total_elapsed_time / idx) * num_images
        estimated_time_left = estimated_total_time - total_elapsed_time

        update_output(f"Time taken for this image: {elapsed_time:.2f}s")
        update_output(f"Elapsed total time: {total_elapsed_time:.2f}s")
        update_output(f"Estimated time left: {estimated_time_left:.2f}s")

        all_results.append({
            'image': image_file,
            **results
        })
        
        # Update progress bar
        if progress_widget is not None:
            progress_widget.value = idx

    if save_results and results_dir:
        save_results_to_file(all_results, results_dir, results_format, output_widget)

    update_output("<b>Processing completed.</b>")

    return all_results

def print_all_results(results, output_widget=None):
    output = ""
    for result in results:
        output += f"<h3>Image: {result['image']}</h3>"
        output += f"<p>Coverage: {result['coverage']:.2f}%</p>"
        output += f"<p>Total number of nuclei: {result['total_nuclei']}</p>"
        output += f"<p>Number of nuclei in myotubes (with at least 3 nuclei): {result['nuclei_in_myotubes']}</p>"
        output += f"<p>Nuclear Fusion Index: {result['nuclear_fusion_index']:.2f}%</p>"
        output += f"<p>Number of myotubes with at least 3 nuclei: {len(result['myotube_results'])}</p>"
        
        # Create a table for myotube data
        if len(result['myotube_results']) > 0:
            output += "<table border='1' style='border-collapse: collapse; width: 100%;'>"
            output += "<tr><th>Myotube ID</th><th>Nuclei count</th><th>Length</th><th>Diameter</th><th>Surface area</th></tr>"
            
            for myotube in result['myotube_results']:
                output += f"<tr><td>{myotube['id']}</td><td>{myotube['nuclei_count']}</td><td>{myotube['length']:.2f}</td><td>{myotube['diameter']:.2f}</td><td>{myotube['surface_area']:.2f}</td></tr>"
            
            output += "</table>"
        output += "<hr>"
    
    if output_widget is not None:
        with output_widget:
            display(HTML(output))
    
    return output

def print_summary_results(results, output_widget=None):
    avg_myotube_count = np.mean([len(r['myotube_results']) for r in results])
    avg_coverage = np.mean([r['coverage'] for r in results])
    avg_nuclear_fusion_index = np.mean([r['nuclear_fusion_index'] for r in results])
    avg_total_nuclei = np.mean([r['total_nuclei'] for r in results])
    avg_nuclei_in_myotubes = np.mean([r['nuclei_in_myotubes'] for r in results])

    output = "<h2>Summary Results</h2>"
    output += f"<p>Average number of myotubes with at least 3 nuclei: {avg_myotube_count:.2f}</p>"
    output += f"<p>Average coverage: {avg_coverage:.2f}%</p>"
    output += f"<p>Average Nuclear Fusion Index: {avg_nuclear_fusion_index:.2f}%</p>"
    output += f"<p>Average total number of nuclei: {avg_total_nuclei:.2f}</p>"
    output += f"<p>Average number of nuclei in myotubes (with at least 3 nuclei): {avg_nuclei_in_myotubes:.2f}</p>"
    
    if output_widget is not None:
        with output_widget:
            display(HTML(output))
    
    return output

def save_results_to_file(results, directory, file_format, output_widget=None):
    # Update output
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    update_output(f"Saving results to {file_format} file...")
    
    # Create a list to store all data
    all_data = []

    for result in results:
        image_name = result['image']
        coverage = result['coverage']
        total_nuclei = result['total_nuclei']
        nuclei_in_myotubes = result['nuclei_in_myotubes']
        nuclear_fusion_index = result['nuclear_fusion_index']
        
        # Calculate averages
        avg_length = np.mean([m['length'] for m in result['myotube_results']]) if result['myotube_results'] else 0
        avg_diameter = np.mean([m['diameter'] for m in result['myotube_results']]) if result['myotube_results'] else 0
        avg_surface_area = np.mean([m['surface_area'] for m in result['myotube_results']]) if result['myotube_results'] else 0
        avg_nuclei_count = np.mean([m['nuclei_count'] for m in result['myotube_results']]) if result['myotube_results'] else 0
        
        # Add summary row with averages
        all_data.append({
            'Row Type': 'Summary',
            'Image': image_name,
            'Myotube': 'Summary',
            'Length': f'Avg: {avg_length:.2f}',
            'Diameter': f'Avg: {avg_diameter:.2f}',
            'Surface Area': f'Avg: {avg_surface_area:.2f}',
            'Nuclei Count': f'Avg: {avg_nuclei_count:.2f}',
            'Coverage (%)': coverage,
            'Total Nuclei': total_nuclei,
            'Nuclei in Myotubes': nuclei_in_myotubes,
            'Nuclear Fusion Index (%)': nuclear_fusion_index
        })
        
        # Add individual myotube rows
        for myotube in result['myotube_results']:
            all_data.append({
                'Row Type': 'Data',
                'Image': image_name,
                'Myotube': myotube['id'],
                'Length': myotube['length'],
                'Diameter': myotube['diameter'],
                'Surface Area': myotube['surface_area'],
                'Nuclei Count': myotube['nuclei_count'],
                'Coverage (%)': '',
                'Total Nuclei': '',
                'Nuclei in Myotubes': '',
                'Nuclear Fusion Index (%)': ''
            })

    # Create a DataFrame
    df = pd.DataFrame(all_data)

    # Determine file extension based on format
    if file_format.lower() == 'excel':
        file_extension = '.xlsx'
    elif file_format.lower() == 'csv':
        file_extension = '.csv'
    else:
        update_output(f"Unsupported file format: {file_format}, defaulting to CSV")
        file_extension = '.csv'

    # Create filename
    filename = f"myotube_analysis_results{file_extension}"
    filepath = os.path.join(directory, filename)

    # Save to file
    try:
        if file_format.lower() == 'excel':
            df.to_excel(filepath, index=False, engine='openpyxl')
            
            # Apply conditional formatting to color summary rows
            workbook = load_workbook(filepath)
            worksheet = workbook.active
            red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
            
            for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=1):
                cell = row[0]
                if cell.value == 'Summary':
                    for cell in row:
                        cell.fill = red_fill
            
            workbook.save(filepath)
            update_output(f"Results saved to {filepath} with colored summary rows")
        else:  # csv
            df.to_csv(filepath, index=False)
            update_output(f"Results saved to {filepath} with 'Row Type' column to indicate summary rows")
    except ImportError:
        update_output("Warning: openpyxl is not installed. Saving results as CSV instead.")
        csv_filepath = os.path.join(directory, "myotube_analysis_results.csv")
        df.to_csv(csv_filepath, index=False)
        update_output(f"Results saved to {csv_filepath} with 'Row Type' column to indicate summary rows")
    except Exception as e:
        update_output(f"An error occurred while saving the file: {str(e)}")
        update_output("Attempting to save as CSV...")
        try:
            csv_filepath = os.path.join(directory, "myotube_analysis_results.csv")
            df.to_csv(csv_filepath, index=False)
            update_output(f"Results saved to {csv_filepath} with 'Row Type' column to indicate summary rows")
        except Exception as e:
            update_output(f"Failed to save results: {str(e)}")



#### Data Saving and Batch Processing

These functions handle results storage and multi-image analysis:

- `save_mask_files`: Exports segmentation masks as 16-bit TIFF files for downstream analysis
- `process_single_image`: Runs the complete analysis pipeline on an individual image
- `process_dataset`: Processes multiple images with progress tracking and time estimation
- `print_all_results`/`print_summary_results`: Generate detailed and statistical reports
- `save_results_to_file`: Exports all metrics to CSV/Excel with summary rows highlighted

These functions enable automated batch processing of large image datasets while maintaining organized outputs for statistical analysis and publication.

### 8. Interactive Widgets

Provide interactive widgets to facilitate user input for analysis parameters.

##### Programmatic Usage (Alternative to UI)

If you prefer to run the analysis directly in a notebook cell rather than using the interactive widgets, you can use the following code:

```python
results = process_dataset(
    directory="path/to/dataset",
    use_gpu=True,
    min_myotube_size=200,
    max_coverage_percent=40,
    minimum_fiber_intensity=25,
    maximum_fiber_intensity=200,
    initial_nuclei_threshold=50,
    max_nuclei_threshold=120,
    save_plots=True,
    plots_dir="path/to/save/plots",
    save_masks=True,
    masks_dir="path/to/save/masks",
    save_results=True,
    results_dir="path/to/save/results",
    results_format="CSV"  # or "Excel"
)

The following function creates a comprehensive widget-based interface for:

- Input/output directory selection and channel specification
- Parameter adjustment for myotube and nuclei detection
- Processing options (GPU usage, cropping)
- Output customization (plots, masks, data export)

The UI organizes parameters into logical sections and handles the complete analysis workflow from input selection to results visualization, making the tool accessible to users without programming experience.

In [ ]:
def create_widgets():
    # Main output widget
    output = Output(
        layout=Layout(width='100%', height='400px', border='1px solid #ddd', overflow_y='auto')
    )
    
    # Progress bar
    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        bar_style='info',
        orientation='horizontal'
    )
    
    # Existing widgets
    input_directory = Text(description='Input Directory:')
    min_myotube_size = IntSlider(description='Min Myotube Size:', min=50, max=500, step=10, value=200)
    max_coverage_percent = FloatText(description='Max Coverage %:', value=40, min=10, max=80, step=1)
    use_gpu = Checkbox(description='Use GPU', value=True)

    # Threshold widgets
    minimum_fiber_intensity = IntSlider(description='Min Fiber Intensity:', min=0, max=100, step=1, value=25)
    maximum_fiber_intensity = IntSlider(description='Max Fiber Intensity:', min=100, max=255, step=1, value=200)
    initial_nuclei_threshold = IntSlider(description='Initial Nuclei Threshold:', min=0, max=100, step=1, value=50)
    max_nuclei_threshold = IntSlider(description='Max Nuclei Threshold:', min=50, max=200, step=1, value=120)
    manual_diameter = FloatText(description='Manual Diameter:', value=17.0, min=1.0, max=100.0, step=0.5)
    use_manual_diameter = Checkbox(description='Use Manual Diameter', value=False)
    use_cropping = Checkbox(description='Use Cropping for Estimation', value=True, tooltip='Crop center portion of image for faster diameter estimation')


    # Saving options
    save_plots = Checkbox(description='Save Plots', value=False)
    plots_directory = Text(description='Plots Directory:')
    save_masks = Checkbox(description='Save Masks', value=False)
    masks_directory = Text(description='Masks Directory:')
    save_results = Checkbox(description='Save Results', value=False)
    results_directory = Text(description='Results Directory:')
    results_format = Dropdown(description='Results Format:', options=['Excel', 'CSV'], value='Excel')

    # Run button
    run_button = Button(description="Run Analysis", button_style='success')

    # Function to update directory widget visibility
    def update_directory_visibility(change):
        if change['owner'] == save_plots:
            plots_directory.layout.display = 'flex' if change['new'] else 'none'
        elif change['owner'] == save_masks:
            masks_directory.layout.display = 'flex' if change['new'] else 'none'
        elif change['owner'] == save_results:
            results_directory.layout.display = 'flex' if change['new'] else 'none'
            results_format.layout.display = 'flex' if change['new'] else 'none'

    # Dropdown for myotube channel selection
    myotube_channel_selector = Dropdown(
    description='Myotube Channel:',
    options=['green', 'red'],
    value='green'  # preselected green channel
    )
    # Set up visibility toggles
    save_plots.observe(update_directory_visibility, names='value')
    save_masks.observe(update_directory_visibility, names='value')
    save_results.observe(update_directory_visibility, names='value')

    # Initially hide directory inputs
    plots_directory.layout.display = 'none'
    masks_directory.layout.display = 'none'
    results_directory.layout.display = 'none'
    results_format.layout.display = 'none'

    def on_button_click(b):
        # Clear previous output
        output.clear_output()
        
        with output:
            display(HTML("<h2>Analysis in progress...</h2>"))
        
        # Reset progress bar
        progress.value = 0
        
        try:
            results = process_dataset(
                input_directory.value,
                use_gpu.value,
                min_myotube_size.value,
                max_coverage_percent.value,
                minimum_fiber_intensity.value,
                maximum_fiber_intensity.value,
                initial_nuclei_threshold.value,
                max_nuclei_threshold.value,
                myotube_channel=myotube_channel_selector.value,  
                save_plots=save_plots.value,
                plots_dir=plots_directory.value if save_plots.value else None,
                save_masks=save_masks.value,
                masks_dir=masks_directory.value if save_masks.value else None,
                save_results=save_results.value,
                results_dir=results_directory.value if save_results.value else None,
                results_format=results_format.value,
                use_manual_diameter=use_manual_diameter.value,
                manual_diameter=manual_diameter.value,
                use_cropping=use_cropping.value,
                output_widget=output,
                progress_widget=progress
            )
            
            # Don't clear output so plots remain visible
            with output:
                display(HTML("<h2>Analysis Complete</h2>"))
                display(HTML("<p>Processinsg finished. Plots are shown above. Results have been saved.</p>"))
            
            # Don't display metrics in the output - they're saved to file
            if save_results and results_directory.value:
                with output:
                    display(HTML(f"<p>All metrics have been saved to: {os.path.join(results_directory.value, 'myotube_analysis_results.' + ('xlsx' if results_format.value.lower() == 'excel' else 'csv'))}</p>"))
            
        except Exception as e:
            output.clear_output()
            with output:
                display(HTML(f"<h2 style='color: red;'>Error during analysis</h2><p>{str(e)}</p>"))

    run_button.on_click(on_button_click)

    # Combine all widgets into a VBox
    parameter_section = VBox([
        input_directory,
        myotube_channel_selector,
        HBox([
            VBox([
                Label("Myotube Detection Parameters"),
                max_coverage_percent,
                min_myotube_size
            ]),
            VBox([
                Label("Intensity Thresholds"),
                minimum_fiber_intensity,
                maximum_fiber_intensity
            ]),
            VBox([
                Label("Nuclei Parameters"),
                initial_nuclei_threshold,
                max_nuclei_threshold,
                manual_diameter,
                use_manual_diameter,
                use_cropping
            ])
        ]),
        HBox([
            VBox([
                Label("Processing Options"),
                use_gpu
            ]),
            VBox([
                Label("Output Options"),
                save_plots,
                plots_directory,
                save_masks,
                masks_directory,
                save_results,
                results_directory,
                results_format
            ])
        ]),
        run_button,
        progress
    ])
    
    all_widgets = VBox([
        parameter_section,
        output
    ])

    # Display the form
    display(HTML("<h1>Myotube Analysis Tool</h1>"))
    display(HTML("<p>This tool analyzes microscopy images to detect myotubes and nuclei, and calculates metrics such as the Nuclear Fusion Index.</p>"))
    display(all_widgets)

create_widgets()